# Recurrent Neural Network (RNN)

## Import dependencies

In [1]:
import numpy as np

## Load dataset

In [6]:
data = open('../datasets/wikitext-2-raw/wiki.train.raw', 'r').read()
chars = sorted(list(set(data)))

In [8]:
data_size = len(data)
char_size = len(chars)
print('Data size = {:,}\nChar size = {:,}'.format(data_size, char_size))

Data size = 10,918,892
Char size = 1,013


### Characters to index and vice versa

In [10]:
char_2_idx = {ch: i for i,ch in enumerate(chars)}
idx_2_char = {i: ch for i,ch in enumerate(chars)}
print('c2i = {:,}\ni2c = {:,}'.format(len(char_2_idx), len(idx_2_char)))

c2i = 1,013
i2c = 1,013


### Testing vectorization

In [16]:
vector_a = np.zeros(shape=[char_size])
idx_4_a = char_2_idx['a']
vector_a[idx_4_a] = 1
print(vector_a)

[ 0.  0.  0. ...,  0.  0.  0.]


## Define Hyperparameters

In [20]:
# Training
hidden_size = 100
seq_length = 25
learning_rate = 1e-1
max_iter = 1000
log_step = 100

# Model parameters
Wxh = np.random.randn(hidden_size, char_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why = np.random.randn(char_size, hidden_size) * 0.01
bh = np.zeros(shape=[hidden_size])
by = np.zeros(shape=[char_size])